In [20]:
# Imports

# Matemaricos, mapiulacao de dados e funcoes
import numpy as np
import pandas as pd
import time
# Desligando alertas de chained_assignment
pd.options.mode.chained_assignment = None  # default='warn'

# Graficos
#import matplotlib
#import matplotlib.pyplot as plt
#import seaborn as sns

# Chamadas e tratamentos de dados
import re
import requests

In [21]:
# Carregamento da listagem de propostas
listagem = pd.DataFrame(pd.read_excel('../Listagem/listagem_13.xlsx'))

In [22]:
# Limpeza de colunas vazias
listagem = listagem.drop(listagem.columns[[1,3,5,7,9]], axis=1)

# Antigo
#listagem = listagem.drop('Unnamed: 1', axis=1)
#listagem = listagem.drop('Unnamed: 3', axis=1)
#listagem = listagem.drop('Unnamed: 5', axis=1)
#listagem = listagem.drop('Unnamed: 7', axis=1)
#listagem = listagem.drop('Unnamed: 9', axis=1)

In [24]:
# Limpeza de linhas varias (NaN) - A SER FEITO
listagem = listagem.dropna()

In [25]:
#Criacao das colunas de codigo do tipo e id do projeto
listagem['Cod_Tipo'] = ''
listagem['Id_Projeto'] = ''
listagem['Data_Projeto'] = ''


In [29]:
# Preenchimento do campo Cod_Tipo
for i in listagem.index:
    #print(listagem["Tipo"][i])
    if(listagem["Tipo"][i] == "Requerimento"):
        listagem['Cod_Tipo'][i] = 'REQ'
    elif(listagem["Tipo"][i] == "Projeto de Lei"):
        listagem['Cod_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Lei Complementar"):
        listagem['Cod_Tipo'][i] = 'PLEIC'
    elif(listagem["Tipo"][i] == "Representação"):
        listagem['Cod_Tipo'][i] = 'PLEI'
    elif(listagem["Tipo"][i] == "Projeto de Resolução"):
        listagem['Cod_Tipo'][i] = 'PRES'
    elif(listagem["Tipo"][i] == "Pedido de Informação"):
        listagem['Cod_Tipo'][i] = 'PEDIF'
    elif(listagem["Tipo"][i] == "Moção"):
        listagem['Cod_Tipo'][i] = 'MOC'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei)"):
        listagem['Cod_Tipo'][i] = 'MSGPL'
    elif(listagem["Tipo"][i] == "Indicação"):
        listagem['Cod_Tipo'][i] = 'IND'
    elif(listagem["Tipo"][i] == "Projeto de Decreto Legislativo"):
        listagem['Cod_Tipo'][i] = 'PDL'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo"):
        listagem['Cod_Tipo'][i] = 'MSGEX'
    elif(listagem["Tipo"][i] == "Mensagem do Executivo (Projeto de Lei Complementar)"):
        listagem['Cod_Tipo'][i] = 'MSGPC'
    elif(listagem["Tipo"][i] == "Prestação de Contas"):
        listagem['Cod_Tipo'][i] = 'PTCTS'
    elif(listagem["Tipo"][i] == "Projeto de Emenda à Lei Orgânica"):
        listagem['Cod_Tipo'][i] = 'PELOR'
    elif(listagem["Tipo"][i] == "Projeto de Lei de Iniciativa Popular"):
        listagem['Cod_Tipo'][i] = 'PLEIP'
    elif(listagem["Tipo"][i] == "Projeto de Resolução - Comissão de Exame Denúncia"):
        listagem['Cod_Tipo'][i] = 'PRCED'
    elif(listagem["Tipo"][i] == "Proposta de Revisão da Lei Orgânica Municipal"):
        listagem['Cod_Tipo'][i] = 'PRLOM'
    elif(listagem["Tipo"][i] == "Proposta de Revisão do Regimento Interno"):
        listagem['Cod_Tipo'][i] = 'PRRI'
    elif(listagem["Tipo"][i] == "Requerimento Popular"):
        listagem['Cod_Tipo'][i] = 'RPO'
    elif(listagem["Tipo"][i] == "Requerimento de CPI"):
        listagem['Cod_Tipo'][i] = 'RCPI'
    else:
        listagem['Cod_Tipo'][i] = 'DESCONHECIDO'

In [ ]:
# Variaveis de requisicao
num_proj = int(listagem["Projeto"][2])
ano_proj = int(listagem["Ano"][2])
tipo_proj = listagem["Cod_Tipo"][2]

# Requisicao para o portal de propostas da Camara
query = {"numero": num_proj, "ano": ano_proj, "tipo": tipo_proj}
#query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)

# Limpeza para pegar o id do projeto dentro da prefeitura
retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')

listagem['Id_Projeto'][2] = id_projeto

In [37]:
# Preenchimento do campo Cod_Tipo
count = 1

while count > 0:
    for i in listagem.index:

        if listagem['Id_Projeto'][i] == '': 

            # Variaveis de requisicao
            num_proj = int(listagem["Projeto"][i])
            ano_proj = int(listagem["Ano"][i])
            tipo_proj = listagem["Cod_Tipo"][i]

            try:
                # Requisicao para o portal de propostas da Camara
                query = {"numero": num_proj, "ano": ano_proj, "tipo": tipo_proj}
                #query = {"numero": 193600, "ano": 2012, "tipo": 'REQ'}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/listapropos.php?', params=query)
            
                # Limpeza para pegar o id do projeto dentro da prefeitura
                retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))
                id_projeto = re.search('PROPOSICAO.PHP(.+?)TARGET=BLANK', retorno).group(1).replace('?NUM=','').replace('"','')
            except:
                id_projeto = ''
            
            listagem['Id_Projeto'][i] = id_projeto

            time.sleep(0.5)
            
            
    count = 0          

    for i in listagem.index:

        if listagem['Id_Projeto'][i] == '':
            count = count + 1


            

In [40]:
# Contagem e conferencia de qual eh o ultimo projeto com id_projeto preenchido 
count = 0
for i in listagem.index:
    
    if listagem['Id_Projeto'][i] == "":
        print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])
        print(count)
        break
    else:
        count = count + 1

In [45]:
# Preenchimento do campo Data_Projeto
count = 1

while count > 0:

    for i in listagem.index:

        if listagem['Data_Projeto'][i] == '': 

            try:
                # Variaveis de requisicao
                id_proj = listagem['Id_Projeto'][i]

                # Requisicao para o portal de propostas da Camara
                query = {"num": id_proj}
                #query = {"numero": 44920}
                resposta = requests.get('https://www.camarajf.mg.gov.br/sal/proposicao.php?', params=query)

                # Limpeza para pegar o id do projeto dentro da prefeitura
                retorno = (''.join(map(chr, resposta.content.upper())).replace(' ', ''))

                try:
                    data_projeto = re.search('PER&IACUTE;ODO:(.+?)A', retorno).group(1)
                except:
                    data_projeto = 'Data nao encontrada'
            except:
                data_projeto = ''

            listagem['Data_Projeto'][i] = data_projeto

            time.sleep(0.5)
            
            
    count = 0          

    for i in listagem.index:

        if listagem['Data_Projeto'][i] == '':
            count = count + 1



In [46]:
count = 0

for i in listagem.index:

    if listagem['Data_Projeto'][i] == '':
        count = count + 1
        
print(count)

0


In [ ]:
# Conferencia de valores
i = 936

print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], 
      listagem['Id_Projeto'][i], listagem['Data_Projeto'][i], listagem['Situação'][i])

#https://www.camarajf.mg.gov.br/sal/proposicao.php?num=45773

In [ ]:
# Conferencia de valores
i = 1290

print(listagem['Projeto'][i], listagem['Cod_Tipo'][i], listagem['Ano'][i], listagem['Id_Projeto'][i])

#https://www.camarajf.mg.gov.br/sal/listapropos.php?codtipo=PELOR&ano=2012&numero=1

In [47]:
display(listagem)

,Projeto,Ano,Tipo,Autor,Ementa,Situação,Cod_Tipo,Id_Projeto,Data_Projeto
0,40520.0,2013.0,Requerimento,Oliveira Tresse,"Solicita urbanização na rua Antônio Celeste, n...",Arquivada,REQ,44205,11/07/2013
2,4028.0,2013.0,Requerimento,Júlio Gasparette,Solicita limpeza nas bocas de lobo no bairro C...,Arquivada,REQ,44107,09/07/2013
4,36840.0,2013.0,Requerimento,Luiz Otávio Fernandes Coelho - Pardal,solicita recapemamento asfáltico na rua Orland...,Em Tramitação,REQ,43978,Data nao encontrada
6,891.0,2013.0,Requerimento,Cido,Solicita o asfaltamento de toidas as ruas da C...,Arquivada,REQ,31106,05/02/2013
8,8890.0,2013.0,Requerimento,Cido,Solicita o asfaltamento de todas as ruas da co...,Arquivada,REQ,27368,05/02/2013
...,...,...,...,...,...,...,...,...,...
2578,4593.0,2013.0,Requerimento,João do Joaninho,"Solicita limpeza do córrego da rua Principal, ...",Arquivada,REQ,45320,29/08/2013
2580,4592.0,2013.0,Requerimento,João do Joaninho,Solicita limpeza do córrego da rua Dona Euzébi...,Arquivada,REQ,45319,29/08/2013
2582,4591.0,2013.0,Requerimento,João do Joaninho,Solicita reconstrução de uma tampa de bueiro n...,Arquivada,REQ,45318,29/08/2013
2584,4590.0,2013.0,Requerimento,João do Joaninho,Solicita patrolamento das estradas de Rosário ...,Arquivada,REQ,45317,29/08/2013


In [ ]:
# Exportacao da listagem para arquivo csv
listagem.to_csv('../ListagemExp/L_2012.csv', mode='a', encoding = 'utf-8', index=False, header=True)
